# Knuck Parse



https://microsoft.github.io/z3guide/docs/theories/Regular%20Expressions/
https://ivanzuzak.info/noam/webapps/regex_simplifier/

- regular expression equality
- refactoring grammars to LADR etc. Push pop collisions? Ambiguity checking
- Implementation / DFA correspondence to 
def 	Re (s, ctx=None)
def 	ReSort (s)
def 	is_re (s)
def 	InRe (s, re
def 	Union (*args)
def 	Intersect (*args)
def 	Plus (re)
def 	Option (re)
def 	Complement (re)
def 	Star (re)
def 	Loop (re, lo, hi=0)
def 	Range (lo, hi, ctx=None
def 	Diff (a, b, ctx=None)
def 	AllChar (regex_sort, ctx=None)


trans(1, Re("a"), 2)
 as observation function.
 trans(1) = Lambda([s], If(InRe(a,s), 2, 3)
 accept table.  accept(1| | accept(2) | accept(3) 
convert tables to regex?

step(s, )
regex = Function("regex", State, ReSort(StringSort()))
If(accept(s), Empty(), Null()) + Sum(trans(s,a) == s1, Concat("a", regex(s1))
regex(state1)

Convert regex to table?

sre parser to z3 equivalent helper.
https://ahelwer.ca/post/2022-01-19-z3-rbac/





In [12]:
from z3 import *

s1 = Re("ab")
s2 = Re(StringVal("ab"))
s3 = Re(Unit(BoolVal(True)))

x,y,z = Consts("x y z", ReSort(StringSort()))
prove(Union(s1, s1) == s1)
prove(Union(x,x) == x)
prove(Intersect(x,Union(x,y)) == x)





proved
proved
proved


# Old

https://www.philipzucker.com/bryzzowski_kat/

- Do other problems
- Tests
- Tower of theories - kleene is idempotent yada yada. Lattice stuff?
- Reflection to z3 regex
- Better calc

At noon, I promised several classmates couple references in Kleene Algebra. I figure I would write a list of KA literature, mainly in applications (yet, still a lot of math :the_horns: ), with some paper about meta-theories at the end :face_with_monocle: . Hopefully it is helpful to others.
Relating Term Rewriting And Kleene Algebra (promised this noon);   Automated Reasoning (promised this noon);   Coq Tactic
Control Flow Verification: abnormal termination;   every program can be written with one while loop;   non-local control flow;   a negative result;   two-valued indicator variable (fixed the previous negative result);  compiler verification;   program schematology;   schematology in Isabelle
Concurrency: Concurrent Kleene Algebra (CKA);   Kleene Algebra with observation (KAO);   CKAO;   Weak concurrent kleene algebra;   Concurrent netkat
Probability: probablistic Kleene Algebra;   probabilistic protocol verification;   automaton model;   probablistic concurrent kleene algebra
Domain/reachablility reasoning and program logic: modal Kleene Algebra;   Hoare Logic;   Incorrectness logic via modal KA;   Incorrectness logic via TopKAT;   general completeness of TopKAT;   Domain completeness of TopKAT
Hypothesis and Complexity: reduction, a general theory of hypothesis elimination;   completeness and decidablity for concrete KAT;   eliminating hypothesis c p = p;   eliminating e = 0;   complexity of several general class of hypothesis;   several smaller class of hypothesis;   the last open hypothesis: commutativity;   our prove for commutativity hypothesis;
Guarded Kleene Algebra With Tests, a efficient fragment: original paper;   final coalgebra and infinite trace completeness;    automata learning;   cyclic proof
Of course, I think most people would agree the best introductory material of KA and its relation to coalgebra is Prof. Sliva's thesis: https://repository.ubn.ru.nl/handle/2066/83205
Notably, I missed many literature in NetKAT :outbox_tray:  (many literature are already discussed during class), non-well-founded proof :repeat: (lead by Anupam Das and Pous), automata learning :teacher: , relation algebra :twisted_rightwards_arrows:, and coalgebra :leftwards_arrow_with_hook: . I have picked the papers based on the interest of people I met during this summer school, and my naive understanding of this field.
The applications and theories about Kleene algebra (and colagebra) are vast. If you have particular interest, I would be more than happy to talk about them and provide more references if possible :blush: . (edited) 

https://www.dropbox.com/scl/fi/ucs2z7hvzwq4h6hlyfqb0/TA-v1.pdf?rlkey=p4khern2r6w7tmgmwtydl61du&e=1&dl=0 lecture notes
https://www.cs.uoregon.edu/research/summerschool/summer24/topics.php#Silva

In [1]:
from knuckledragger import lemma, axiom
from knuckledragger.kernel import is_proof
from z3 import *


K = DeclareSort("K")
o = Const("o", K)
l = Const("l", K)
char = Function("char", StringSort(), K) # injective btw
a = char(StringVal("a"))
b = char(StringVal("b"))
seq = Function("seq", K,K,K)
par = Function("par1", K,K,K)
star = Function("star", K,K)

ExprRef.__mul__ = lambda x,y: seq(x,y)
ExprRef.__add__ = lambda x,y: par(x,y)
ExprRef.__le__ = lambda x,y: x + y == y

sig = [K, o, l, seq, par, star]

x,y,z,e,f = Consts("x y z e f", K)

#  kozen axioms
# from egglog https://github.com/philzook58/egg-smol/blob/scratchpad2/tests/kleene.egg
par_assoc = axiom(ForAll([x,y,z], x + (y + z) == (x + y) + z))
par_comm = axiom(ForAll([x,y], x + y == y + x))
par_idem = axiom(ForAll([x], x + x == x))
par_zero = axiom(ForAll([x], x + o == x))

seq_assoc = axiom(ForAll([x,y,z], x * (y * z) == (x * y) * z))
seq_zero = axiom(ForAll([x], x * o == o))
seq_one = axiom(ForAll([x], x * l == x))

rdistrib = axiom(ForAll([x,y,z], x * (y + z) == x * y + x * z))
ldistrib = axiom(ForAll([x,y,z], (y + z) * x == y * x + z * x))

unfold = axiom(ForAll([e], star(e) == l + e * star(e)))

# If a set shrinks, star(e) is less than it
least_fix = axiom(ForAll([x,e,f], Implies(f + e * x <= x, star(e) * f <= x)))

kleene = [par_assoc, par_comm, par_idem, par_zero, 
    seq_assoc,  seq_zero, seq_one, 
    ldistrib, rdistrib, unfold, least_fix] 



In [7]:
print(lemma_tptp(ForAll([x],star(x) * star(x) == star(x)), by=kleene, sig=sig, command=["twee"]).stdout.decode())

Here is the input problem:
  Axiom 1 (flattening): star2 = star(x).
  Axiom 2 (hyp): par1(X, X) = X.
  Axiom 3 (hyp): par1(X, Y) = par1(Y, X).
  Axiom 4 (hyp): par1(X, o) = X.
  Axiom 5 (hyp): seq(X, l) = X.
  Axiom 6 (hyp): seq(X, o) = o.
  Axiom 7 (types6): X = X.
  Axiom 8 (ifeq_axiom): ifeq(X, X, Y, Z) = Y.
  Axiom 9 (hyp): par1(X, par1(Y, Z)) = par1(par1(X, Y), Z).
  Axiom 10 (flattening): seq2 = seq(star(x), star(x)).
  Axiom 11 (hyp): seq(X, seq(Y, Z)) = seq(seq(X, Y), Z).
  Axiom 12 (types5): ifeq(X, Y, Z, W) = ifeq(X, Y, Z, W).
  Axiom 13 (types4): ifeq(X, Y, Z, W) = ifeq(X, Y, Z, W).
  Axiom 14 (types3): ifeq(X, Y, Z, W) = ifeq(X, Y, Z, W).
  Axiom 15 (types2): ifeq(X, Y, Z, W) = ifeq(X, Y, Z, W).
  Axiom 16 (hyp): star(X) = par1(l, seq(X, star(X))).
  Axiom 17 (types1): ifeq(X, Y, Z, W) = ifeq(X, Y, Z, W).
  Axiom 18 (hyp): seq(X, par1(Y, Z)) = par1(seq(X, Y), seq(X, Z)).
  Axiom 19 (hyp): seq(par1(X, Y), Z) = par1(seq(X, Z), seq(Y, Z)).
  Axiom 20 (hyp): ifeq(par1(par1(X, s

In [2]:
from knuckledragger.utils import lemma_smt, lemma_tptp
#print("\n".join(lemma_smt(ForAll([x],star(star(x)) == star(x)), by=kleene, sig=sig)))
lemma_tptp(ForAll([x],star(star(x)) == star(x)), by=kleene, sig=sig, command=["vampire"])


CompletedProcess(args=['vampire', '/tmp/temp.p'], returncode=0, stdout=b'% Running in auto input_syntax mode. Trying TPTP\n% Refutation found. Thanks to Tanya!\n% SZS status Theorem for temp\n% SZS output start Proof for temp\n1. ! [X0 : k,X1 : k,X2 : k] : par1(X2,par1(X1,X0)) = par1(par1(X2,X1),X0) [input]\n2. ! [X1 : k,X2 : k] : par1(X2,X1) = par1(X1,X2) [input]\n3. ! [X2 : k] : par1(X2,X2) = X2 [input]\n4. ! [X2 : k] : par1(X2,o) = X2 [input]\n7. ! [X2 : k] : seq(X2,l) = X2 [input]\n9. ! [X0 : k,X1 : k,X2 : k] : seq(X2,par1(X1,X0)) = par1(seq(X2,X1),seq(X2,X0)) [input]\n10. ! [X3 : k] : star(X3) = par1(l,seq(X3,star(X3))) [input]\n11. ! [X4 : k,X3 : k,X2 : k] : (par1(par1(X4,seq(X3,X2)),X2) = X2 => par1(seq(star(X3),X4),X2) = X2) [input]\n12. ! [X2 : k] : star(X2) = star(star(X2)) [input]\n13. ~! [X2 : k] : star(X2) = star(star(X2)) [negated conjecture 12]\n14. ! [X0 : k,X1 : k] : par1(X1,X0) = par1(X0,X1) [rectify 2]\n15. ! [X0 : k] : par1(X0,X0) = X0 [rectify 3]\n16. ! [X0 : k] : 

In [4]:
print("\n".join(lemma_tptp(ForAll([x],star(star(x)) == star(x)), by=kleene, sig=sig)))

tff(sig, type, k : $tType).
tff(sig, type, o : k).
tff(sig, type, l : k).
tff(sig, type, seq : (k * k) > k).
tff(sig, type, par1 : (k * k) > k).
tff(sig, type, star : (k) > k).
tff(hyp, axiom, (![Z:k, Y:k, X:k] : (par1(X, par1(Y, Z)) = par1(par1(X, Y), Z))) ).
tff(hyp, axiom, (![Y:k, X:k] : (par1(X, Y) = par1(Y, X))) ).
tff(hyp, axiom, (![X:k] : (par1(X, X) = X)) ).
tff(hyp, axiom, (![X:k] : (par1(X, o) = X)) ).
tff(hyp, axiom, (![Z:k, Y:k, X:k] : (seq(X, seq(Y, Z)) = seq(seq(X, Y), Z))) ).
tff(hyp, axiom, (![X:k] : (seq(X, o) = o)) ).
tff(hyp, axiom, (![X:k] : (seq(X, l) = X)) ).
tff(hyp, axiom, (![Z:k, Y:k, X:k] : (seq(par1(Y, Z), X) = par1(seq(Y, X), seq(Z, X)))) ).
tff(hyp, axiom, (![Z:k, Y:k, X:k] : (seq(X, par1(Y, Z)) = par1(seq(X, Y), seq(X, Z)))) ).
tff(hyp, axiom, (![E:k] : (star(E) = par1(l, seq(E, star(E))))) ).
tff(hyp, axiom, (![F:k, E:k, X:k] : ((par1(par1(F, seq(E, X)), X) = X) => (par1(seq(star(E), F), X) = X))) ).
tff( conjecture,(![X:k] : (star(star(X)) = star(X))) ).

In [20]:
%%file /tmp/kleene.smt2
(set-logic UF)
;;declarations
(declare-sort K 0)
(declare-fun o () K)
(declare-fun l () K)
(declare-fun seq (K K) K)
(declare-fun par1 (K K) K)
(declare-fun star (K) K)
;;axioms
(assert (forall ((x K) (y K) (z K)) (= (par1 x (par1 y z)) (par1 (par1 x y) z))))
(assert (forall ((x K) (y K)) (= (par1 x y) (par1 y x))))
(assert (forall ((x K)) (= (par1 x x) x)))
(assert (forall ((x K)) (= (par1 x o) x)))
(assert (forall ((x K) (y K) (z K)) (= (seq x (seq y z)) (seq (seq x y) z))))
(assert (forall ((x K)) (= (seq x o) o)))
(assert (forall ((x K)) (= (seq x l) x)))
(assert (forall ((x K) (y K) (z K)) (= (seq (par1 y z) x) (par1 (seq y x) (seq z x)))))
(assert (forall ((x K) (y K) (z K)) (= (seq x (par1 y z)) (par1 (seq x y) (seq x z)))))
(assert (forall ((e K)) (= (star e) (par1 l (seq e (star e))))))
(assert (forall ((x K) (e K) (f K))
  (let ((a!1 (= (par1 (par1 f (seq e x)) x) x))
        (a!2 (= (par1 (seq (star e) f) x) x)))
    (=> a!1 a!2))))
;;goal
(assert (let ((a!1 (forall ((x K)) (= (star (star x)) (star x)))))
  (not a!1)))
(check-sat)

Overwriting /tmp/kleene.smt2


In [21]:
! vampire  --mode smtcomp  /tmp/kleene.smt2

% Running in auto input_syntax mode. Trying SMTLIB2
% Exception at proof search level
User error: This version cannot be used with this logic!


In [19]:
!z3 /tmp/kleene.smt2

^C


In [83]:
def open_binder(l: z3.QuantifierRef):
    vars = []
    nvars = l.num_vars()
    for i in range(nvars):
        sort = l.var_sort(i)
        vars.append(
            (z3.Var(nvars - i - 1, sort), z3.FreshConst(sort, prefix=l.var_name(i)))
        )
    return [x for (_, x) in vars], z3.substitute(l.body(), vars)




In [3]:


#print("\n".join(lemma_tptp(ForAll([x],star(star(x)) == star(x)), by=kleene, sig=sig)))
lemma_tptp(ForAll([x],star(star(x)) == star(x)), by=kleene, sig=sig, command=["vampire"])

TimeoutExpired: Command '['vampire', '/tmp/temp.p']' timed out after 1.0 seconds

In [77]:
%%file /tmp/kleene.p
tff(sig, type, sK : $tType).
tff(sig, type, o : sK).
tff(sig, type, l : sK).
tff(sig, type, seq : (sK * sK) > sK).
tff(sig, type, par1 : (sK * sK) > sK).
tff(sig, type, star : (sK) > sK).
tff(hyp, axiom, (![Z:sK, Y:sK, X:sK] : (par1(X, par1(Y, Z)) = par1(par1(X, Y), Z))) ).
tff(hyp, axiom, (![Y:sK, X:sK] : (par1(X, Y) = par1(Y, X))) ).
tff(hyp, axiom, (![X:sK] : (par1(X, X) = X)) ).
tff(hyp, axiom, (![X:sK] : (par1(X, o) = X)) ).
tff(hyp, axiom, (![Z:sK, Y:sK, X:sK] : (seq(X, seq(Y, Z)) = seq(seq(X, Y), Z))) ).
tff(hyp, axiom, (![X:sK] : (seq(X, o) = o)) ).
tff(hyp, axiom, (![X:sK] : (seq(X, l) = X)) ).
tff(hyp, axiom, (![Z:sK, Y:sK, X:sK] : (seq(par1(Y, Z), X) = par1(seq(Y, X), seq(Z, X)))) ).
tff(hyp, axiom, (![Z:sK, Y:sK, X:sK] : (seq(X, par1(Y, Z)) = par1(seq(X, Y), seq(X, Z)))) ).
tff(hyp, axiom, (![E:sK] : (star(E) = par1(l, seq(E, star(E))))) ).
tff(hyp, axiom, (![F:sK, E:sK, X:sK] : ((par1(par1(F, seq(E, X)), X) = X) => (par1(seq(star(E), F), X) = X))) ).
tff(goal, conjecture,(![X:sK] : (star(star(X)) = star(X))) ).

Overwriting /tmp/kleene.p


In [81]:
!twee /tmp/kleene.p

4190. par1(star3, seq(x, seq(star2, par1(star2, X)))) -> par1(star3, seq(x, seq(star2, X)))
4191. par1(star3, seq(x, seq(star2, seq(x, x)))) -> star3
4192. seq(star2, seq(par1(X, seq(star2, x)), star2)) -> seq(star2, seq(par1(X, x), star2))
4193. seq(star3, seq(par1(X, seq(star3, x)), star2)) -> seq(star3, seq(par1(X, x), star2))
4194. seq(par1(x, seq(par1(l, X), star3)), star2) -> par1(star2, seq(par1(X, l), star3))
4195. par1(star2, seq(star(seq(x, star3)), star(X))) -> seq(star(seq(x, star3)), star(X))
4196. par1(star2, star(star(star(seq(x, star3))))) -> star(star(star(seq(x, star3))))
4197. seq(par1(x, seq(l, par1(l, X))), star3) -> seq(par1(star3, seq(l, X)), star3)
4198. seq(par1(X, star(l)), seq(X, star(X))) -> seq(star(X), seq(X, star(X)))
4199. seq(par1(x, star(l)), seq(x, star2)) -> seq(star2, seq(x, star2))
4200. seq(o, seq(star2, seq(par1(star2, X), star2))) -> seq(o, seq(star2, seq(X, star2)))
4201. par1(X, par1(X, seq(Y, seq(star(Z), W)))) -> par1(X, seq(Y, seq(star(Z), 